In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [30]:
dftr=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotrain.csv')
dfte=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotest.csv')

In [31]:
dftecp=dfte.copy()

In [32]:
dftr.drop(['id','program_id','trainee_id'],1,inplace=True)
dfte.drop(['id','program_id','trainee_id'],1,inplace=True)

In [33]:
dftr.reset_index(drop=True,inplace=True)
dfte.reset_index(drop=True,inplace=True)

In [34]:
dfte['is_pass']=np.nan

In [35]:
print(dftr.shape,dfte.shape)

(73147, 13) (31349, 13)


In [36]:
dftr['data']='train'
dfte['data']='test'
dff=pd.concat([dftr,dfte],axis=0)

In [37]:
# dff is now our merged dataframe with train and test data both.

In [38]:
# for i in dftr.select_dtypes(include='object').columns:
#     print(i,'\n',dff[i].value_counts(),'\n')

In [39]:
dff.isnull().sum()

program_type                     0
program_duration                 0
test_id                          0
test_type                        0
difficulty_level                 0
gender                           0
education                        0
city_tier                        0
age                          39520
total_programs_enrolled          0
is_handicapped                   0
trainee_engagement_rating      108
is_pass                      31349
data                             0
dtype: int64

In [40]:
dff['education'].unique()

array(['Matriculation', 'High School Diploma', 'Bachelors', 'Masters',
       'No Qualification'], dtype=object)

In [41]:
# dff[(dff['is_pass']==1) & (dff['total_programs_enrolled']>1) 
#     & (dff['trainee_engagement_rating']>3) 
#     & (dff['education']=='No Qualification')]['age'].value_counts()

In [42]:

dff1=dff.copy() # copy before making any changes or replacement in your data.this is backup

In [43]:
# dff=dff1.copy()
# dff[dff['data']=='train']['trainee_engagement_rating'].median()

In [44]:
dff.loc[(dff['trainee_engagement_rating'].isnull()) & 
    (    (dff['education']=='High School Diploma') | (dff['education']=='Matriculation')   ) 
        &(dff['program_duration']==134),['trainee_engagement_rating']]=2

dff.loc[(dff['trainee_engagement_rating'].isnull())
   & (dff['is_pass']==0),['trainee_engagement_rating']]=1

# dff[(dff['trainee_engagement_rating'].isnull()) & (dff['is_pass']==1)]
dff.loc[(dff['trainee_engagement_rating'].isnull()) & (dff['is_pass']==1) 
    & (dff['test_type']=='online') & (dff['difficulty_level']=='easy'),['trainee_engagement_rating']]=2


dff.loc[(dff['trainee_engagement_rating'].isnull()) & (dff['test_type']=='online')
        & (dff['program_type']=='T'),['trainee_engagement_rating']]=1

dff.loc[dff['trainee_engagement_rating'].isnull(),['trainee_engagement_rating']]=2

In [45]:
# dff[(dff['trainee_engagement_rating'].isnull()) & (dff['test_type']=='online')]['program_type']


In [46]:
dff['age'].replace({np.nan:30},inplace=True) # imputed by median
# dff['trainee_engagement_rating'].replace({np.nan:1},inplace=True)# imputed by median

In [47]:
dff.select_dtypes(include='object').columns

Index(['program_type', 'test_type', 'difficulty_level', 'gender', 'education',
       'is_handicapped', 'data'],
      dtype='object')

In [48]:
dff['difficulty_level'].replace({'vary hard':'very hard'},1,True)

dff['difficulty_level'].replace({'easy':0,'intermediate':1,'hard':2,'very hard':3},1,True)

dff['education'].replace({'No Qualification':0,
                          'Matriculation':1,
                         'High School Diploma':2,
                         'Bachelors':3,
                         'Masters':4},1,True)

# conversion from days to months:
dff['program_duration']=np.round((dff['program_duration']/30),2)

In [49]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [50]:
for i in dff.select_dtypes(include='object'):
    if i!='data':
        dff[i]=le.fit_transform(dff[i])

In [51]:
dff.head()

,program_type,program_duration,test_id,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass,data
0,5,4.53,150,0,2,1,3,3,24.0,5,0,1.0,0.0,train
1,1,4.37,44,0,0,0,1,4,26.0,2,0,3.0,1.0,train
2,6,4.00,178,1,0,1,3,1,40.0,1,0,2.0,1.0,train
3,1,3.90,32,0,0,0,3,3,30.0,4,0,1.0,1.0,train
4,3,4.37,84,0,2,0,1,1,42.0,2,0,4.0,1.0,train


In [52]:
# putting train and test into their respective original dataframes

In [53]:
dftr=dff[dff['data']=='train'].drop('data',1)

dfte=dff[dff['data']=='test'].drop(['data','is_pass'],1)

In [54]:
# delete dff
del dff

In [55]:
# Now lets begin building machine learning model

In [56]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [57]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']

In [58]:
from sklearn.model_selection import train_test_split

# 1-Logistic regression:

In [59]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,log_loss,roc_auc_score

In [60]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver = 'liblinear')

logreg.fit(X_train, y_train)

y_prob = logreg.predict_proba(X_test)
print(y_prob[:,1]) 

y_pred = logreg.predict(X_test)
print(y_pred)
      
print(accuracy_score(y_test, y_pred))

## For test data 

y_prob_train = logreg.predict_proba(X_train)
y_prob_train[:,1]

y_pred_train = logreg.predict(X_train)
y_pred_train

y_prob_test = logreg.predict_proba(X_test)
y_prob_test[:,1]

y_pred_test = logreg.predict(X_test)
y_pred_test

print('Confusion Matrix - Train:', '\n', confusion_matrix(y_train, y_pred_train))
print('Overall Accuracy', accuracy_score(y_train, y_pred_train))

print('Confusion Matrix - Test:', '\n', confusion_matrix(y_test, y_pred_test))
print('Overall Accuracy', accuracy_score(y_test, y_pred_test))
      
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test,y_prob))

print('roc_auc score',roc_auc_score(y_test,y_pred_test))

[0.56279683 0.79206325 0.59812146 ... 0.5103801  0.8917783  0.79027552]
[1. 1. 1. ... 1. 1. 1.]
0.7069036226930964
Confusion Matrix - Train: 
 [[ 2150 13510]
 [ 1503 34039]]
Overall Accuracy 0.706788797312605
Confusion Matrix - Test: 
 [[  836  5784]
 [  648 14677]]
Overall Accuracy 0.7069036226930964
log loss:  0.577453569060502
roc_auc score 0.542000068998487


### DT hyperparam tuning:

In [61]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
rscv = RandomizedSearchCV(dt, dt_params, cv=5, scoring='roc_auc')
rscv.fit(X, y)
print(rscv.best_params_)

{'min_samples_leaf': 26, 'max_depth': 9, 'criterion': 'gini'}


In [62]:
rscv_best_DT=rscv.best_params_
DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X_train,y_train)
y_pred_dt=DT.predict(X_test)
roc_auc_score(y_test,y_pred_dt)

0.6058787696584083

### Bagged model tuning:

In [63]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

In [64]:
# from sklearn.model_selection import KFold
# model = BaggingClassifier()
# n_estimators = [10, 100]
# # define grid search
# grid = dict(n_estimators=n_estimators)
# cv = KFold(n_splits=5,random_state=0)
# grid_search = RandomizedSearchCV(estimator=model, param_distributions=grid, cv=cv, scoring='roc_auc',error_score=0)
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

### Random Forest  tuning:

In [65]:
# model = RandomForestClassifier()
# n_estimators = [10, 100, 200]
# max_features = ['sqrt', 'log2']
# # define grid search
# grid = dict(n_estimators=n_estimators,max_features=max_features)
# cv = KFold(n_splits=5, random_state=0)
# grid_search = RandomizedSearchCV(estimator=model, param_distributions=grid,cv=cv, scoring='roc_auc')
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [66]:
dt = DecisionTreeClassifier(**rscv_best_DT)

In [67]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =1)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier(**{'n_estimators': 200, 'max_features': 'log2','random_state':0})
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier(n_estimators=100)
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier(**rscv_best_DT)
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,Random Forest,0.725862,0.946818,0.716655,0.773530,0.837898
1,Bagging,0.724956,0.946818,0.715607,0.778994,0.825187
2,Decistion Tree,0.710761,0.730518,0.720027,0.741831,0.916328
3,Ada boost,0.705646,0.714874,0.716792,0.733688,0.930546
4,KNN,0.682848,0.795906,0.704261,0.755446,0.849889
5,bagged LR,0.666765,0.704816,0.709319,0.717305,0.960621
6,Logistic Regression,0.666693,0.704972,0.709091,0.717087,0.960752
7,Naive-Bayes,0.652565,0.679895,0.683390,0.745077,0.828070


# Random Forest:[ Best model ]:

In [68]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
rf=RandomForestClassifier(**{'n_estimators': 200, 'max_features': 'log2','random_state':0})

In [70]:
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [71]:
ypredrf=rf.predict_proba(dfte)[:,1]

In [72]:
dftetmp=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotest.csv')

dftetmp=pd.DataFrame(dftetmp.iloc[:,0])

ypredrf=pd.DataFrame(ypredrf,columns=['is_pass'])

final=pd.concat([dftetmp,ypredrf],1)

final.to_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/finalrf.csv',index=False)

# Bagged Classifier is giving 2nd highest output

In [58]:
from sklearn.model_selection import KFold,cross_val_score

In [60]:
from sklearn.ensemble import BaggingClassifier
X=dftr.drop('is_pass',1)
y=dftr['is_pass']

In [61]:
from sklearn.ensemble import BaggingClassifier
bgc=BaggingClassifier(n_estimators=100)

In [62]:
bgc.fit(X,ya)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=100,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [63]:
ypred=bgc.predict(dfte)

dftetmp=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotest.csv')

dftetmp=pd.DataFrame(dftetmp.iloc[:,0])

ypred=pd.DataFrame(ypred,columns=['is_pass']).astype('int')

final=pd.concat([dftetmp,ypred],1)

final.to_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/final.csv',index=False)

In [135]:
# import scipy.stats as st

# 6.47-6.07

# se=.4/np.sqrt(499)

# .4/se

# st.t.isf(0.025,df=1) # 22 is greater than 12,its in critical region,so,reject null

# st.t.sf(22.33,1) # p less than alpha reject null,its diff in length

# Testing Part 

### All using label encoding [ Check the accuracy ]-part 1

In [64]:
dff2=dff1.copy()

In [65]:
dff1.head()

,program_type,program_duration,test_id,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass,data
0,Y,136,150,offline,intermediate,M,Matriculation,3,24.0,5,N,1.0,0.0,train
1,T,131,44,offline,easy,F,High School Diploma,4,26.0,2,N,3.0,1.0,train
2,Z,120,178,online,easy,M,Matriculation,1,40.0,1,N,2.0,1.0,train
3,T,117,32,offline,easy,F,Matriculation,3,NaN,4,N,1.0,1.0,train
4,V,131,84,offline,intermediate,F,High School Diploma,1,42.0,2,N,4.0,1.0,train


In [66]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [139]:
# dff11.select_dtypes(include='object')
dff1['age'].replace({np.nan:28},inplace=True) # imputed by median
dff1['trainee_engagement_rating'].replace({np.nan:2},inplace=True)# imputed by median

In [140]:
for i in dff1.select_dtypes(include='object'):
    if i!='data':
        dff1[i]=le.fit_transform(dff1[i])

In [141]:
# dff1 is now fully transformed into label encoded[auto]

In [142]:
dftr1=dff1[dff1['data']=='train'].drop('data',1)

dfte1=dff1[dff1['data']=='test'].drop(['data','is_pass'],1)

In [143]:
# dftr1=dftr1.astype('int')
# dfte1=dfte1.astype('int')

In [144]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=dftr1.drop('is_pass',1)
y=dftr1['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier(random_state=0)
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier(n_estimators=100)
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
# bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            ac=metrics.roc_auc_score(y_test,y_test_pred)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree'
                                ],
                       'auc_score':auc,'training':sctr,'testing':scte})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,auc_score,names,testing,training
0,0.639272,Bagging,0.712554,0.950393
1,0.638925,Random Forest,0.717521,0.950412
2,0.621742,Decistion Tree,0.669674,0.950412
3,0.606891,KNN,0.700524,0.795965
4,0.586211,Naive-Bayes,0.675416,0.680950
5,0.575050,Ada boost,0.717111,0.713468
6,0.541545,Logistic Regression,0.706448,0.706047


## All using get_dummies:

In [145]:
dff3=dff2.copy()

In [146]:
# dff11.select_dtypes(include='object')
dff2['age'].replace({np.nan:39},inplace=True) # imputed by median
dff2['trainee_engagement_rating'].replace({np.nan:2},inplace=True)# imputed by median

In [147]:
dff2.select_dtypes(include='object').columns.tolist()

['program_type',
 'test_type',
 'difficulty_level',
 'gender',
 'education',
 'is_handicapped',
 'data']

In [148]:
dff2.select_dtypes(include='object').nunique().sum()-7

17

In [149]:
dff2=pd.get_dummies(dff2,prefix=['program_type',
 'test_type',
 'difficulty_level',
 'gender',
 'education',
 'is_handicapped','data'],drop_first=1)

In [150]:
# dff3.head(1)

In [151]:
# dff2.replace()

In [152]:
dftr2=dff2[dff2['data_train']==1].drop('data_train',1)
dfte2=dff2[dff2['data_train']==0].drop(['data_train','is_pass'],1)

In [153]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=dftr2.drop('is_pass',1)
y=dftr2['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier(random_state=0)
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier(n_estimators=100)
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
# bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            ac=metrics.roc_auc_score(y_test,y_test_pred)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree'
                                ],
                       'auc_score':auc,'training':sctr,'testing':scte})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,auc_score,names,testing,training
0,0.641393,Bagging,0.714377,0.950412
1,0.637537,Random Forest,0.716063,0.950412
2,0.616932,Decistion Tree,0.667031,0.950412
3,0.608998,KNN,0.703167,0.797527
4,0.596757,Naive-Bayes,0.669173,0.669407
5,0.581896,Ada boost,0.718524,0.715910
6,0.564763,Logistic Regression,0.711552,0.708937


# Using XGB:

In [154]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [155]:
# With original data :[mixed manual label and one hot]

In [156]:
xgb=XGBClassifier()

X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)


xgb.fit(X_train,y_train)

y_xgb=xgb.predict(X_test)

roc_auc_score(y_test,y_xgb)

0.5915527616644407

In [157]:
# with all label encoded data:

In [158]:
xgb=XGBClassifier()

X=dftr1.drop('is_pass',1)
y=dftr1['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)
xgb.fit(X_train,y_train)
y_xgb=xgb.predict(X_test)
roc_auc_score(y_test,y_xgb)

0.5914374109796307

In [159]:
# with all get_dummies data.

In [160]:
xgb=XGBClassifier()

X=dftr2.drop('is_pass',1)
y=dftr2['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

xgb.fit(X_train,y_train)

y_xgb2=xgb.predict(X_test)

roc_auc_score(y_test,y_xgb2)

0.5948518750338833

# Using RFE:

In [161]:
from sklearn.svm import SVC

In [162]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

In [163]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
# create the RFE model for the svm classifier 
# and select attributes
model=LogisticRegression()
rfe = RFE(model, 3)
rfe = rfe.fit(X,y)
# print summaries for the selection of attributes
print(rfe.support_)
print(rfe.ranking_)
print('Optimal number of features: {}'.format(rfe.n_features_))

[False False False  True False False False False False False  True  True]
[ 8  7  9  1  5  2  3  4 10  6  1  1]
Optimal number of features: 3


In [164]:
print(np.where(rfe.support_ == False)[0])

[0 1 2 4 5 6 7 8 9]


In [165]:
X.drop(X.columns[np.where(rfe.support_ == False)[0]], axis=1).head()

,test_type,is_handicapped,trainee_engagement_rating
0,0,0,1.0
1,0,0,3.0
2,1,0,2.0
3,0,0,1.0
4,0,0,4.0


 	test_type 	is_handicapped 	trainee_engagement_rating

In [166]:
X=dftr.drop(columns=['test_type', 'is_handicapped', 'trainee_engagement_rating'])
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

In [167]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics

X=dftr.drop(columns=['test_type', 'is_handicapped', 'trainee_engagement_rating','is_pass'])
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =0)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier(random_state=0)
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            ac=metrics.roc_auc_score(y_test,y_test_pred)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,auc_score,names,testing,training
0,0.599483,Random Forest,0.689998,0.892817
1,0.596217,Bagging,0.673092,0.883442
2,0.590602,Decistion Tree,0.647573,0.892836
3,0.586579,KNN,0.687674,0.775712
4,0.543241,Ada boost,0.704443,0.702219
5,0.538366,Naive-Bayes,0.697334,0.693078
6,0.511607,bagged LR,0.700706,0.697531
7,0.510927,Logistic Regression,0.700296,0.697453


# Cleaning the data and and trying again :(  

In [168]:
del dff4

NameError: name 'dff4' is not defined

In [ ]:
dff4=dff3.copy()

In [ ]:
dff4.head(1)

In [ ]:
# dff4['difficulty_level'].replace({'vary hard':'very hard'},1,True)

# dff4['difficulty_level'].replace({'easy':0,'intermediate':1,'hard':2,'very hard':3},1,True)

# dff4['education'].replace({'No Qualification':0,
#                           'Matriculation':1,
#                          'High School Diploma':2,
#                          'Bachelors':3,
#                          'Masters':4},1,True)

dff['New_Price'] = dff.groupby('brand').New_Price.apply(lambda x: x.fillna(x.mean()))
dff.New_Price = dff.New_Price.fillna(dff.New_Price.mean())

In [ ]:
import lightgbm as lgb 

In [ ]:
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.3)

In [ ]:
train_data=lgb.Dataset(X,y)
param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param['metric'] = ['auc']

In [ ]:
lgbm=lgb.train(param,train_data,num_round)

In [ ]:
ypred2=lgbm.predict(x_test)

In [ ]:
auc_xgb =  roc_auc_score(y_test,ypred2)
auc_xgb

In [ ]:
ypredlgb=lgbm.predict(dfte)

In [ ]:
ypredlgb=lgbm.predict(dfte)

dftetmp=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotest.csv')

dftetmp=pd.DataFrame(dftetmp.iloc[:,0])

ypredlgb=pd.DataFrame(ypredlgb,columns=['is_pass']).astype('int')

final=pd.concat([dftetmp,ypredlgb],1)

final.to_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/lgbfinal.csv',index=False)

In [ ]:
from lightgbm import LGBMClassifier
# define dataset
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.3)
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# evaluate the model
model = LGBMClassifier()
model.fit(x_train, y_train)
yhat = model.predict(x_test)

In [ ]:
yhat.shape

In [ ]:
x_test.shape

In [ ]:
roc_auc_score(y_test,yhat)

In [86]:
from sklearn.ensemble import BaggingClassifier
X=dftr.drop('is_pass',1)
y=dftr['is_pass']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =1)
from sklearn.ensemble import BaggingClassifier
bgc=BaggingClassifier(n_estimators=100)
bgc.fit(X,y)
# bgc=ensemble.BaggingClassifier(n_estimators=100)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=100,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [87]:
ypred=bgc.predict(dfte)

dftetmp=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfotest.csv')

dftetmp=pd.DataFrame(dftetmp.iloc[:,0])

ypred=pd.DataFrame(ypred,columns=['is_pass']).astype('int')

final=pd.concat([dftetmp,ypred],1)

final.to_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/final.csv',index=False)

# Other method Cat GBM

In [15]:
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

In [16]:
dftr=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfodftr.csv')
dfte=pd.read_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/perfodfte.csv')


dftr.fillna({"age": dftr["age"].median(), "dftree_engagement_rating": dftr["dftree_engagement_rating"].mode()[0]},inplace=True)
dfte.fillna({"age": dfte["age"].median(), "dftree_engagement_rating": dfte["dftree_engagement_rating"].mode()[0]},inplace=True)

In [17]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

dftr['program_id'] = le.fit_transform(dftr['program_id'])
dftr['program_type'] = le.fit_transform(dftr['program_type'])
dftr['dfte_type'] = le.fit_transform(dftr['dfte_type'])
dftr['difficulty_level'] = le.fit_transform(dftr['difficulty_level'])
dftr['gender'] = le.fit_transform(dftr['gender'])
dftr['education'] = le.fit_transform(dftr['education'])
dftr['is_handicapped'] = le.fit_transform(dftr['is_handicapped'])

dfte['program_id'] = le.fit_transform(dfte['program_id'])
dfte['program_type'] = le.fit_transform(dfte['program_type'])
dfte['dfte_type'] = le.fit_transform(dfte['dfte_type'])
dfte['difficulty_level'] = le.fit_transform(dfte['difficulty_level'])
dfte['gender'] = le.fit_transform(dfte['gender'])
dfte['education'] = le.fit_transform(dfte['education'])
dfte['is_handicapped'] = le.fit_transform(dfte['is_handicapped'])

In [18]:
feature_names = [x for x in train.columns if x not in ['id', 'is_pass']]
target = train['is_pass']

In [20]:
vote_est = [
    ('cat1',CatBoostClassifier(depth=12, iterations=100, learning_rate=0.1, eval_metric='AUC', random_seed=42,verbose=False)),
    ('ca2',CatBoostClassifier(depth=4, iterations=100, learning_rate=0.1, eval_metric='AUC', random_seed=42,verbose=False)),
    ('cat3',CatBoostClassifier(depth=8, iterations=100, learning_rate=0.1, eval_metric='AUC', random_seed=42,verbose=False))
]

model = VotingClassifier(estimators = vote_est , voting = 'soft')
## model training and prediction
model.fit(train[feature_names], target)
pred = model.predict_proba(test[feature_names])

In [21]:
## make submission
sub = pd.DataFrame()
sub['id'] = test['id']
sub['is_pass'] = [i[1] for i in pred]
sub.to_csv('D:/datasets+minipro/analytics vidhya datasets/MLSPhack/result.csv', index=False)